In [1]:
import json
import matplotlib.pyplot as plt
import numpy as np
from mpl_toolkits.mplot3d import Axes3D
from tqdm import tqdm_notebook
%matplotlib inline

In [43]:
def load_data(input_data):
    """Загрузка входных данных из файла"""
    #with open(file, 'r') as f:
    #    input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for i, depotData in enumerate(input_data['depots']):
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[orderData['pickup_point_id']] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points

In [126]:
import os
basedir = "/home/ksihkun/git_projects/phystech"
example_dir = os.path.join(basedir, "data")
input_file =  os.path.join(example_dir, "contest_input.json")

In [127]:
#data = load_data(input_file)
with open(input_file, 'r') as f:
    input_data = json.load(f)

In [128]:
couriers, orders, points = load_data(input_data)

TypeError: expected str, bytes or os.PathLike object, not dict

In [129]:
calc_1 = 0
calc_2 = 0
calc_3 = 0
calc_4 = 0
calc_5 = 0
bad_ids = []
for key in orders:
    if orders[key]['pickup_from'] > orders[key]['dropoff_to']:
        calc_1+=1
        bad_ids.append(key)
    if orders[key]['pickup_to'] < (60*6):
        calc_2+=1
        bad_ids.append(key)
    if orders[key]['dropoff_to'] < (60*6):
        calc_3+=1
        bad_ids.append(key)
    time_travel = get_travel_duration_minutes([orders[key]['pickup_location_x'],orders[key]['pickup_location_y']],[orders[key]['dropoff_location_x'],orders[key]['dropoff_location_y']], )
    if 15+ time_travel>(-orders[key]['pickup_from'] + orders[key]['dropoff_to']):
        calc_4+=1
        bad_ids.append(key)
    if orders[key]['payment'] < 50:
        calc_5+=1
print(calc_1, calc_2, calc_3, calc_4, calc_5)
print(len(set(bad_ids)))

0 75 6 1908 3
1934


In [130]:
filtered_orders = {}
for key in orders:
    if key not in bad_ids:
        filtered_orders[key] = orders[key]

In [131]:
orders = filtered_orders

In [132]:
data = [couriers, orders, points ]

In [133]:
def get_travel_duration_minutes(location1, location2):
    """Время перемещения курьера от точки location1 до точки location2 вминутах"""
    distance = abs(location1[0] - location2[0]) + abs(location1[1] - location2[1])
    return 10 + distance

In [134]:
def get_current_courier_reward(time, payment):
    work_payment = time * 2
    profit = orders_payment - work_payment
    return profit

In [55]:
len(data[1])

5369

In [9]:
payments.sum()

2836613

In [10]:
len(payments)

7303

In [11]:
couriers_start_position[0]

[111, 5]

In [135]:
def delete_element(array, idx):
    result = []
    for i in range(len(array)):
        if not(i == idx):
            result.append(array[i])
    return result

In [13]:
type([1,2,3,4])

list

In [14]:
type(delete_element([1,2,3,6], 0))

list

In [67]:
data[1][i+10001]

{'order_id': 10001,
 'pickup_point_id': 40001,
 'pickup_from': 480,
 'pickup_to': 570,
 'pickup_location_x': 284,
 'pickup_location_y': 235,
 'dropoff_point_id': 60001,
 'dropoff_from': 600,
 'dropoff_to': 960,
 'dropoff_location_x': 252,
 'dropoff_location_y': 197,
 'payment': 313}

In [68]:
i= 0 
np.array([[data[1][i+10001]['pickup_location_x'],
                           data[1][key]['pickup_location_y']] for key in data[1]])

array([[284, 235],
       [284, 141],
       [284, 108],
       ...,
       [284, 163],
       [284, 108],
       [284, 115]])

In [137]:
payment = 0
reward = 0
stop_flag = 0
i = 0

couriers_start_position = [data[0][i+1]['location'] for i in range(len(data[0]))]
points_pickup = np.array([[data[1][i+10001]['pickup_location_x'],
                           data[1][key]['pickup_location_y']] for key in data[1]])
time_pickup = np.array([[data[1][10001+i]['pickup_from'],
                         data[1][key]['pickup_to']] for key in data[1]])
time_dropoff = np.array([[data[1][10001+i]['dropoff_from'],
                          data[1][key]['dropoff_to']] for  key in data[1]])    
points_dropoff = np.array([[data[1][i+10001]['dropoff_location_x'],
                            data[1][key]['dropoff_location_y']] for key in data[1]])
payments = np.array([data[1][key]['payment'] for key in data[1]])
pickup_id = np.array([data[1][key]['pickup_point_id'] for key in data[1]])
dropoff_id = np.array([data[1][key]['dropoff_point_id'] for key in data[1]])
cnt = 0
cnt_courier = 0
routes = []
for courier in tqdm_notebook(couriers_start_position):
    if stop_flag == 1:
        break 
    print('courier {} begins route at {}'.format(cnt_courier, courier))
    cnt_courier += 1
    tmp_x = [courier[0]]
    tmp_y = [courier[1]]
    print('processing courier {}..\n'.format(cnt))
    cnt += 1
    courier_position = courier
    current_time = 360
    distances = []
    routes = []
    # select avaliable pickup points
    while current_time < 1400:
        distances = []
        avaliable_pickups = []
        print("current time {}. Passing order {}\n".format(current_time, cnt))
        cnt += 1
        tmp_route = []
        for j in range(len(points_pickup)):
            tmp_route.append( {"courier_id": str(cnt_courier),
                              "action":"pickup", "order_id":str(10001+j),
                              "point_id":str(pickup_id[j])})
            tmp_route.append( {"courier_id": str(cnt_courier),
                              "action":"dropoff", "order_id":str(10001+j),
                              "point_id":str(dropoff_id[j])})
            # calculate route_time for each point from current courier position to dropoff
            route_time = get_travel_duration_minutes(points_pickup[j],points_dropoff[j]) + get_travel_duration_minutes(points_pickup[j],
                                                                                             courier_position)
            if current_time in time_pickup[j] and current_time + route_time:
                avaliable_pickups.append(points_pickup[j])       
            
        # calculate Manhattan distance to closest avaliable points
        min_dist = np.inf
        if len(avaliable_pickups) == 0:
            break
            print('сука')
        for i in range(len(avaliable_pickups)):
            dist = get_travel_duration_minutes(courier, avaliable_pickups[i])
            if not(dist == 0):
                tmp = get_travel_duration_minutes(courier, avaliable_pickups[i])
                if tmp < min_dist:
                    min_dist = tmp
                    current_order = i
                    
        # delete visited location

        print('the most far point: {}'.format(min_dist))
        courier_position = points_dropoff[j]
        print('send courier on {}'.format(courier_position))
        tmp_x.append(courier_position[0])
        tmp_y.append(courier_position[1])
        
        current_time += route_time
        payment = payments[current_order] 
        print('Payment: {}, salary: {}'.format(payments[current_order], route_time))
        
        #deleting visited elements
        points_dropoff = delete_element(points_dropoff,current_order)
        time_pickup = delete_element(time_pickup,current_order)
        points_pickup = delete_element(points_pickup, current_order)
        time_dropoff = delete_element(time_dropoff, current_order)
        payments = delete_element(payments, current_order)
    
        print("{} points left".format(len(points_pickup)))
    reward += payment   
    print(reward)
    print("-"*80)
    routes.append(tmp_route)
print('total reward: {}'.format(reward - (current_time-360)*2))




courier 0 begins route at [111, 5]
processing courier 0..

current time 360. Passing order 1

the most far point: 227
send courier on [252  51]
Payment: 155, salary: 399
5368 points left
current time 759. Passing order 2

155
--------------------------------------------------------------------------------
courier 1 begins route at [260, 112]
processing courier 3..

current time 360. Passing order 4

the most far point: 36
send courier on [252  51]
Payment: 244, salary: 143
5367 points left
current time 503. Passing order 5

399
--------------------------------------------------------------------------------
courier 2 begins route at [207, 66]
processing courier 6..

current time 360. Passing order 7

the most far point: 93
send courier on [252  51]
Payment: 380, salary: 242
5366 points left
current time 602. Passing order 8

779
--------------------------------------------------------------------------------
courier 3 begins route at [288, 55]
processing courier 9..

current time 360. 

the most far point: 86
send courier on [252  51]
Payment: 291, salary: 224
5338 points left
current time 584. Passing order 82

the most far point: 170
send courier on [252  51]
Payment: 310, salary: 212
5337 points left
current time 796. Passing order 83

9611
--------------------------------------------------------------------------------
courier 26 begins route at [295, 15]
processing courier 84..

current time 360. Passing order 85

the most far point: 55
send courier on [252  51]
Payment: 547, salary: 227
5336 points left
current time 587. Passing order 86

the most far point: 232
send courier on [252  51]
Payment: 372, salary: 212
5335 points left
current time 799. Passing order 87

the most far point: 198
send courier on [252  51]
Payment: 260, salary: 212
5334 points left
current time 1011. Passing order 88

9871
--------------------------------------------------------------------------------
courier 27 begins route at [364, 158]
processing courier 89..

current time 360. Passi

18052
--------------------------------------------------------------------------------
courier 50 begins route at [125, 126]
processing courier 164..

current time 360. Passing order 165

the most far point: 170
send courier on [252  51]
Payment: 338, salary: 286
5304 points left
current time 646. Passing order 166

18390
--------------------------------------------------------------------------------
courier 51 begins route at [204, 98]
processing courier 167..

current time 360. Passing order 168

the most far point: 92
send courier on [252  51]
Payment: 183, salary: 213
5303 points left
current time 573. Passing order 169

18573
--------------------------------------------------------------------------------
courier 52 begins route at [59, 179]
processing courier 170..

current time 360. Passing order 171

the most far point: 250
send courier on [252  51]
Payment: 210, salary: 405
5302 points left
current time 765. Passing order 172

the most far point: 297
send courier on [252  51]

27830
--------------------------------------------------------------------------------
courier 75 begins route at [277, 244]
processing courier 246..

current time 360. Passing order 247

the most far point: 43
send courier on [252  51]
Payment: 459, salary: 252
5272 points left
current time 612. Passing order 248

the most far point: 36
send courier on [252  51]
Payment: 345, salary: 212
5271 points left
current time 824. Passing order 249

28175
--------------------------------------------------------------------------------
courier 76 begins route at [291, 125]
processing courier 250..

current time 360. Passing order 251

the most far point: 17
send courier on [252  51]
Payment: 383, salary: 133
5270 points left
current time 493. Passing order 252

28558
--------------------------------------------------------------------------------
courier 77 begins route at [266, 56]
processing courier 253..

current time 360. Passing order 254

the most far point: 32
send courier on [252  51]
P

35151
--------------------------------------------------------------------------------
courier 96 begins route at [165, 134]
processing courier 321..

current time 360. Passing order 322

the most far point: 131
send courier on [252  51]
Payment: 386, salary: 254
5239 points left
current time 614. Passing order 323

35537
--------------------------------------------------------------------------------
courier 97 begins route at [291, 114]
processing courier 324..

current time 360. Passing order 325

the most far point: 17
send courier on [252  51]
Payment: 419, salary: 124
5238 points left
current time 484. Passing order 326

the most far point: 50
send courier on [252  51]
Payment: 242, salary: 212
5237 points left
current time 696. Passing order 327

35779
--------------------------------------------------------------------------------
courier 98 begins route at [162, 158]
processing courier 328..

current time 360. Passing order 329

the most far point: 142
send courier on [252  51

...

the most far point: 107
send courier on [252  51]
Payment: 391, salary: 212
5118 points left
current time 771. Passing order 632

the most far point: 153
send courier on [252  51]
Payment: 290, salary: 212
5117 points left
current time 983. Passing order 633

67466
--------------------------------------------------------------------------------
courier 191 begins route at [166, 201]
processing courier 634..

current time 360. Passing order 635

the most far point: 135
send courier on [252  51]
Payment: 250, salary: 320
5116 points left
current time 680. Passing order 636

67716
--------------------------------------------------------------------------------
courier 192 begins route at [163, 111]
processing courier 637..

current time 360. Passing order 638

the most far point: 134
send courier on [252  51]
Payment: 568, salary: 241
5115 points left
current time 601. Passing order 639

68284
--------------------------------------------------------------------------------
courier 193 beg

75494
--------------------------------------------------------------------------------
courier 215 begins route at [184, 174]
processing courier 711..

current time 360. Passing order 712

the most far point: 130
send courier on [252  51]
Payment: 250, salary: 275
5087 points left
current time 635. Passing order 713

75744
--------------------------------------------------------------------------------
courier 216 begins route at [270, 211]
processing courier 714..

current time 360. Passing order 715

the most far point: 41
send courier on [252  51]
Payment: 374, salary: 226
5086 points left
current time 586. Passing order 716

76118
--------------------------------------------------------------------------------
courier 217 begins route at [106, 150]
processing courier 717..

current time 360. Passing order 718

the most far point: 190
send courier on [252  51]
Payment: 290, salary: 329
5085 points left
current time 689. Passing order 719

76408
--------------------------------------

the most far point: 190
send courier on [252  51]
Payment: 524, salary: 329
5057 points left
current time 689. Passing order 787

82703
--------------------------------------------------------------------------------
courier 238 begins route at [235, 132]
processing courier 788..

current time 360. Passing order 789

the most far point: 63
send courier on [252  51]
Payment: 250, salary: 182
5056 points left
current time 542. Passing order 790

82953
--------------------------------------------------------------------------------
courier 239 begins route at [231, 133]
processing courier 791..

current time 360. Passing order 792

the most far point: 66
send courier on [252  51]
Payment: 404, salary: 187
5055 points left
current time 547. Passing order 793

83357
--------------------------------------------------------------------------------
courier 240 begins route at [200, 46]
processing courier 794..

current time 360. Passing order 795

the most far point: 97
send courier on [252  5

90520
--------------------------------------------------------------------------------
courier 260 begins route at [44, 92]
processing courier 866..

current time 360. Passing order 867

the most far point: 258
send courier on [252  51]
Payment: 108, salary: 379
5022 points left
current time 739. Passing order 868

the most far point: 325
send courier on [252  51]
Payment: 147, salary: 212
5021 points left
current time 951. Passing order 869

90667
--------------------------------------------------------------------------------
courier 261 begins route at [259, 142]
processing courier 870..

current time 360. Passing order 871

the most far point: 35
send courier on [252  51]
Payment: 321, salary: 168
5020 points left
current time 528. Passing order 872

90988
--------------------------------------------------------------------------------
courier 262 begins route at [123, 25]
processing courier 873..

current time 360. Passing order 874

the most far point: 195
send courier on [252  5

97590
--------------------------------------------------------------------------------
courier 282 begins route at [319, 113]
processing courier 944..

current time 360. Passing order 945

the most far point: 46
send courier on [252  51]
Payment: 370, salary: 153
4988 points left
current time 513. Passing order 946

the most far point: 71
send courier on [252  51]
Payment: 302, salary: 212
4987 points left
current time 725. Passing order 947

the most far point: 64
send courier on [252  51]
Payment: 208, salary: 212
4986 points left
current time 937. Passing order 948

97798
--------------------------------------------------------------------------------
courier 283 begins route at [228, 95]
processing courier 949..

current time 360. Passing order 950

the most far point: 71
send courier on [252  51]
Payment: 250, salary: 192
4985 points left
current time 552. Passing order 951

the most far point: 73
send courier on [252  51]
Payment: 169, salary: 212
4984 points left
current time 76

In [138]:
routes = routes[0]

In [139]:
routes

[{'courier_id': '300',
  'action': 'pickup',
  'order_id': '10001',
  'point_id': '40001'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10001',
  'point_id': '60001'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10002',
  'point_id': '40003'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10002',
  'point_id': '60003'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10003',
  'point_id': '40005'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10003',
  'point_id': '60005'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10004',
  'point_id': '40008'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10004',
  'point_id': '60008'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10005',
  'point_id': '40010'},
 {'courier_id': '300',
  'action': 'dropoff',
  'order_id': '10005',
  'point_id': '60010'},
 {'courier_id': '300',
  'action': 'pickup',
  'order_id': '10006',
  'poin

In [140]:
routes_dict = dict({})

In [141]:
for i in range(len(routes)):
    routes_dict.update({str(i):routes[i]})

In [142]:
data=json.dumps(routes_dict)

In [143]:
with open('output.json', 'w') as outfile:
    json.dump(data, outfile)

In [144]:
with open('output.json', 'r') as outfile:
    data = json.load(outfile)
output_data = json.loads(data)

In [101]:
for event in output_data:
    print(event, output_data[event])

0 {'courier_id': '300', 'action': 'pickup', 'order_id': '10001', 'point_id': '40001'}
1 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10001', 'point_id': '60001'}
2 {'courier_id': '300', 'action': 'pickup', 'order_id': '10002', 'point_id': '40003'}
3 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10002', 'point_id': '60003'}
4 {'courier_id': '300', 'action': 'pickup', 'order_id': '10003', 'point_id': '40005'}
5 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10003', 'point_id': '60005'}
6 {'courier_id': '300', 'action': 'pickup', 'order_id': '10004', 'point_id': '40008'}
7 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10004', 'point_id': '60008'}
8 {'courier_id': '300', 'action': 'pickup', 'order_id': '10005', 'point_id': '40010'}
9 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10005', 'point_id': '60010'}
10 {'courier_id': '300', 'action': 'pickup', 'order_id': '10006', 'point_id': '40011'}
11 {'courier_id': '300', 'action': 'dropoff', 'o

1121 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10561', 'point_id': '60918'}
1122 {'courier_id': '300', 'action': 'pickup', 'order_id': '10562', 'point_id': '40919'}
1123 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10562', 'point_id': '60919'}
1124 {'courier_id': '300', 'action': 'pickup', 'order_id': '10563', 'point_id': '40925'}
1125 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10563', 'point_id': '60925'}
1126 {'courier_id': '300', 'action': 'pickup', 'order_id': '10564', 'point_id': '40929'}
1127 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10564', 'point_id': '60929'}
1128 {'courier_id': '300', 'action': 'pickup', 'order_id': '10565', 'point_id': '40930'}
1129 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10565', 'point_id': '60930'}
1130 {'courier_id': '300', 'action': 'pickup', 'order_id': '10566', 'point_id': '40931'}
1131 {'courier_id': '300', 'action': 'dropoff', 'order_id': '10566', 'point_id': '60931'}
1132 {'courier_

2371 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11186', 'point_id': '62002'}
2372 {'courier_id': '300', 'action': 'pickup', 'order_id': '11187', 'point_id': '42004'}
2373 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11187', 'point_id': '62004'}
2374 {'courier_id': '300', 'action': 'pickup', 'order_id': '11188', 'point_id': '42006'}
2375 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11188', 'point_id': '62006'}
2376 {'courier_id': '300', 'action': 'pickup', 'order_id': '11189', 'point_id': '42007'}
2377 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11189', 'point_id': '62007'}
2378 {'courier_id': '300', 'action': 'pickup', 'order_id': '11190', 'point_id': '42008'}
2379 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11190', 'point_id': '62008'}
2380 {'courier_id': '300', 'action': 'pickup', 'order_id': '11191', 'point_id': '42010'}
2381 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11191', 'point_id': '62010'}
2382 {'courier_

3888 {'courier_id': '300', 'action': 'pickup', 'order_id': '11945', 'point_id': '43281'}
3889 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11945', 'point_id': '63281'}
3890 {'courier_id': '300', 'action': 'pickup', 'order_id': '11946', 'point_id': '43283'}
3891 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11946', 'point_id': '63283'}
3892 {'courier_id': '300', 'action': 'pickup', 'order_id': '11947', 'point_id': '43284'}
3893 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11947', 'point_id': '63284'}
3894 {'courier_id': '300', 'action': 'pickup', 'order_id': '11948', 'point_id': '43285'}
3895 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11948', 'point_id': '63285'}
3896 {'courier_id': '300', 'action': 'pickup', 'order_id': '11949', 'point_id': '43287'}
3897 {'courier_id': '300', 'action': 'dropoff', 'order_id': '11949', 'point_id': '63287'}
3898 {'courier_id': '300', 'action': 'pickup', 'order_id': '11950', 'point_id': '43289'}
3899 {'courier_i

5620 {'courier_id': '300', 'action': 'pickup', 'order_id': '12811', 'point_id': '44683'}
5621 {'courier_id': '300', 'action': 'dropoff', 'order_id': '12811', 'point_id': '64683'}
5622 {'courier_id': '300', 'action': 'pickup', 'order_id': '12812', 'point_id': '44684'}
5623 {'courier_id': '300', 'action': 'dropoff', 'order_id': '12812', 'point_id': '64684'}
5624 {'courier_id': '300', 'action': 'pickup', 'order_id': '12813', 'point_id': '44685'}
5625 {'courier_id': '300', 'action': 'dropoff', 'order_id': '12813', 'point_id': '64685'}
5626 {'courier_id': '300', 'action': 'pickup', 'order_id': '12814', 'point_id': '44686'}
5627 {'courier_id': '300', 'action': 'dropoff', 'order_id': '12814', 'point_id': '64686'}
5628 {'courier_id': '300', 'action': 'pickup', 'order_id': '12815', 'point_id': '44687'}
5629 {'courier_id': '300', 'action': 'dropoff', 'order_id': '12815', 'point_id': '64687'}
5630 {'courier_id': '300', 'action': 'pickup', 'order_id': '12816', 'point_id': '44688'}
5631 {'courier_i

7370 {'courier_id': '300', 'action': 'pickup', 'order_id': '13686', 'point_id': '45620'}
7371 {'courier_id': '300', 'action': 'dropoff', 'order_id': '13686', 'point_id': '65620'}
7372 {'courier_id': '300', 'action': 'pickup', 'order_id': '13687', 'point_id': '45621'}
7373 {'courier_id': '300', 'action': 'dropoff', 'order_id': '13687', 'point_id': '65621'}
7374 {'courier_id': '300', 'action': 'pickup', 'order_id': '13688', 'point_id': '45622'}
7375 {'courier_id': '300', 'action': 'dropoff', 'order_id': '13688', 'point_id': '65622'}
7376 {'courier_id': '300', 'action': 'pickup', 'order_id': '13689', 'point_id': '45623'}
7377 {'courier_id': '300', 'action': 'dropoff', 'order_id': '13689', 'point_id': '65623'}
7378 {'courier_id': '300', 'action': 'pickup', 'order_id': '13690', 'point_id': '45624'}
7379 {'courier_id': '300', 'action': 'dropoff', 'order_id': '13690', 'point_id': '65624'}
7380 {'courier_id': '300', 'action': 'pickup', 'order_id': '13691', 'point_id': '45625'}
7381 {'courier_i

9120 {'courier_id': '300', 'action': 'pickup', 'order_id': '14561', 'point_id': '46495'}
9121 {'courier_id': '300', 'action': 'dropoff', 'order_id': '14561', 'point_id': '66495'}
9122 {'courier_id': '300', 'action': 'pickup', 'order_id': '14562', 'point_id': '46496'}
9123 {'courier_id': '300', 'action': 'dropoff', 'order_id': '14562', 'point_id': '66496'}
9124 {'courier_id': '300', 'action': 'pickup', 'order_id': '14563', 'point_id': '46497'}
9125 {'courier_id': '300', 'action': 'dropoff', 'order_id': '14563', 'point_id': '66497'}
9126 {'courier_id': '300', 'action': 'pickup', 'order_id': '14564', 'point_id': '46498'}
9127 {'courier_id': '300', 'action': 'dropoff', 'order_id': '14564', 'point_id': '66498'}
9128 {'courier_id': '300', 'action': 'pickup', 'order_id': '14565', 'point_id': '46499'}
9129 {'courier_id': '300', 'action': 'dropoff', 'order_id': '14565', 'point_id': '66499'}
9130 {'courier_id': '300', 'action': 'pickup', 'order_id': '14566', 'point_id': '46500'}
9131 {'courier_i

In [158]:
def main(input_data, output_data):
    couriers, orders, points = load_data(input_data)
    for step, event in enumerate(output_data):
        
        courier_id = output_data[event]['courier_id']
        action = output_data[event]['action']
        order_id = output_data[event]['order_id']
        point_id = output_data[event]['point_id']
        print(courier_id)
        courier = couriers[int(courier_id)]
        point = points[int(point_id)]
        order = orders[int(order_id)]

        # Последнее местоположение курьера
        courier_location = courier['location']
        # Местоположение точки назначения, куда направляется курьер
        destination_location = point['location']
        # Время перемещения до точки назначения
        duration_minutes = get_travel_duration_minutes(courier_location, destination_location)
        # Самое раннее время, в которое курьер может оказаться на точке назначения
        visit_time = courier['time'] + duration_minutes

        if visit_time < point['timewindow'][0]:
            # Если курьер прибывает раньше левой границы временного интервала на точке, то он ждет начала интервала
            visit_time = point['timewindow'][0]
        elif visit_time > point['timewindow'][1]:
            # Если курьер прибывает позже правой границы временного интервала на точке, то это опоздание
            print(step, event)
            #raise Exception('Courier will be late')

        if action == 'pickup':
            # Если order_id сейчас не в точке point_id, то ошибка
            if 'order_time' not in point or order_id not in point['order_time']:
                print(step, event)
                #raise Exception('Cant pickup')
            # Если курьер едет за заказом на склад, то, возможно, ему нужно будет подождать появления этого заказа на складе
            if is_depot_point(point_id) and visit_time < point['order_time'][order_id]:
                visit_time = point['order_time'][order_id]
            # Курьер забрал заказ, удаляем информацию о том, с какого времени заказ находится в этой точке
            point.pop('order_time', None)
        elif action == 'dropoff':
            # Если point_id, не id склада или id точки dropoff заказа order_id, то ошибка (курьер привез заказ не туда)
            if not is_depot_point(point_id) and (point_id != order['dropoff_point_id']):
                print(step, event)
                #raise Exception('Cant dropoff')

            # Добавляем информацию о времени появления заказа на точке
            point['order_time'] = {}
            point['order_time'][order_id] = visit_time
        else:
            print(step, event)
            #raise Exception('Unknown action')

        # Обновляем время и местоположение курьера
        courier['time'] = visit_time
        courier['location'] = destination_location

        print('{}. Courier #{} {} order #{} at point #{} at time {}'.format(step, courier_id, action, order_id, point_id, visit_time))

    print('Routes ok')

    # Проверяем, что курьеры выполнили все заказы, которые взяли
    # И посчитаем общую стоимость выполненных заказов
    has_unfinished_orders = False
    orders_payment = 0
    for order_id, order in orders.items():
        pickup_point_id = order['pickup_point_id']
        dropoff_point_id = order['dropoff_point_id']
        if 'order_time' in points[dropoff_point_id] and order_id in points[dropoff_point_id]['order_time']:
            orders_payment += order['payment']
            print('Order #{} completed'.format(order_id))
        elif 'order_time' in points[pickup_point_id] and order_id in points[pickup_point_id]['order_time']:
            print('Order #{} unassigned'.format(order_id))
        else:
            print('Order #{} unfinished'.format(order_id))
            has_unfinished_orders = True

    if has_unfinished_orders:
        #raise Exception('Not all started orders are completed')
        pass
    print('Orders ok')

    # Считаем общую продолжительность работы курьера в минутах
    work_duration = sum([x['time'] - 360 for x in couriers.values()])
    work_payment = work_duration * 2
    profit = orders_payment - work_payment

    print('Total orders payment: {}'.format(orders_payment))
    print('Total couriers payment: {}'.format(work_payment))
    print('Profit: {}'.format(profit))


def load_data(file):
    """Загрузка входных данных из файла"""
    with open(file, 'r') as f:
        input_data = json.load(f)
    couriers = {}
    orders = {}
    points = {}
    for depotData in input_data['depots']:
        points[depotData['point_id']] = {
            'location': [depotData['location_x'], depotData['location_y']],
            'timewindow': [0, 1439],
        }
    for courierData in input_data['couriers']:
        couriers[courierData['courier_id']] = {
            'location': [courierData['location_x'], courierData['location_y']],
            'time': 360,
        }
    for orderData in input_data['orders']:
        points[int(orderData['pickup_point_id'])] = {
            'location': [orderData['pickup_location_x'], orderData['pickup_location_y']],
            'timewindow': [orderData['pickup_from'], orderData['pickup_to']],
            'order_time': {orderData['order_id']: orderData['pickup_from']}
        }
        points[orderData['dropoff_point_id']] = {
            'location': [orderData['dropoff_location_x'], orderData['dropoff_location_y']],
            'timewindow': [orderData['dropoff_from'], orderData['dropoff_to']],
        }
        orders[orderData['order_id']] = orderData
    return couriers, orders, points


def get_travel_duration_minutes(location1, location2):
    """Время перемещения курьера от точки location1 до точки location2 вминутах"""
    distance = abs(location1[0] - location2[0]) + abs(location1[1] - location2[1])
    return 10 + distance


def is_depot_point(point_id):
    """Является ли $pointId точкой склада"""
    return 30001 <= int(point_id) <= 40000

In [159]:
input_file =  os.path.join(example_dir, "contest_input.json")
couriers, orders, points = load_data(input_file)
main(input_file, output_data)

300
0 0
0. Courier #300 pickup order #10001 at point #40001 at time 538
300
1 1
1. Courier #300 dropoff order #10001 at point #60001 at time 618
300
2 2
2. Courier #300 pickup order #10002 at point #40003 at time 711
300
3 3
3. Courier #300 dropoff order #10002 at point #60003 at time 1050
300
4 4
4 4
4. Courier #300 pickup order #10003 at point #40005 at time 1248
300
5 5
5 5
5. Courier #300 dropoff order #10003 at point #60005 at time 1486
300
6 6
6 6
6. Courier #300 pickup order #10004 at point #40008 at time 1827
300
7 7
7 7
7. Courier #300 dropoff order #10004 at point #60008 at time 1976
300
8 8
8 8
8. Courier #300 pickup order #10005 at point #40010 at time 2129
300
9 9
9 9
9. Courier #300 dropoff order #10005 at point #60010 at time 2340
300
10 10
10 10
10. Courier #300 pickup order #10006 at point #40011 at time 2447
300
11 11
11 11
11. Courier #300 dropoff order #10006 at point #60011 at time 2646
300
12 12
12 12
12. Courier #300 pickup order #10007 at point #40012 at time 27

453 453
453. Courier #300 dropoff order #10227 at point #60318 at time 61877
300
454 454
454 454
454. Courier #300 pickup order #10228 at point #40319 at time 61930
300
455 455
455 455
455. Courier #300 dropoff order #10228 at point #60319 at time 61954
300
456 456
456 456
456. Courier #300 pickup order #10229 at point #40320 at time 62026
300
457 457
457 457
457. Courier #300 dropoff order #10229 at point #60320 at time 62125
300
458 458
458 458
458. Courier #300 pickup order #10230 at point #40322 at time 62224
300
459 459
459 459
459. Courier #300 dropoff order #10230 at point #60322 at time 62323
300
460 460
460 460
460. Courier #300 pickup order #10231 at point #40323 at time 62492
300
461 461
461 461
461. Courier #300 dropoff order #10231 at point #60323 at time 62685
300
462 462
462 462
462. Courier #300 pickup order #10232 at point #40324 at time 62775
300
463 463
463 463
463. Courier #300 dropoff order #10232 at point #60324 at time 62801
300
464 464
464 464
464. Courier #300 

300
919 919
919 919
919. Courier #300 dropoff order #10460 at point #60731 at time 124549
300
920 920
920 920
920. Courier #300 pickup order #10461 at point #40732 at time 124774
300
921 921
921 921
921. Courier #300 dropoff order #10461 at point #60732 at time 124908
300
922 922
922 922
922. Courier #300 pickup order #10462 at point #40735 at time 125116
300
923 923
923 923
923. Courier #300 dropoff order #10462 at point #60735 at time 125245
300
924 924
924 924
924. Courier #300 pickup order #10463 at point #40737 at time 125371
300
925 925
925 925
925. Courier #300 dropoff order #10463 at point #60737 at time 125535
300
926 926
926 926
926. Courier #300 pickup order #10464 at point #40740 at time 125682
300
927 927
927 927
927. Courier #300 dropoff order #10464 at point #60740 at time 126028
300
928 928
928 928
928. Courier #300 pickup order #10465 at point #40741 at time 126191
300
929 929
929 929
929. Courier #300 dropoff order #10465 at point #60741 at time 126328
300
930 930
930

1440 1440
1440 1440
1440. Courier #300 pickup order #10721 at point #41210 at time 197355
300
1441 1441
1441 1441
1441. Courier #300 dropoff order #10721 at point #61210 at time 197404
300
1442 1442
1442 1442
1442. Courier #300 pickup order #10722 at point #41212 at time 197583
300
1443 1443
1443 1443
1443. Courier #300 dropoff order #10722 at point #61212 at time 197816
300
1444 1444
1444 1444
1444. Courier #300 pickup order #10723 at point #41213 at time 198012
300
1445 1445
1445 1445
1445. Courier #300 dropoff order #10723 at point #61213 at time 198170
300
1446 1446
1446 1446
1446. Courier #300 pickup order #10724 at point #41214 at time 198270
300
1447 1447
1447 1447
1447. Courier #300 dropoff order #10724 at point #61214 at time 198429
300
1448 1448
1448 1448
1448. Courier #300 pickup order #10725 at point #41215 at time 198606
300
1449 1449
1449 1449
1449. Courier #300 dropoff order #10725 at point #61215 at time 198904
300
1450 1450
1450 1450
1450. Courier #300 pickup order #10

1991 1991
1991. Courier #300 dropoff order #10996 at point #61670 at time 272469
300
1992 1992
1992 1992
1992. Courier #300 pickup order #10997 at point #41671 at time 272766
300
1993 1993
1993 1993
1993. Courier #300 dropoff order #10997 at point #61671 at time 272794
300
1994 1994
1994 1994
1994. Courier #300 pickup order #10998 at point #41672 at time 272839
300
1995 1995
1995 1995
1995. Courier #300 dropoff order #10998 at point #61672 at time 272931
300
1996 1996
1996 1996
1996. Courier #300 pickup order #10999 at point #41674 at time 273068
300
1997 1997
1997 1997
1997. Courier #300 dropoff order #10999 at point #61674 at time 273201
300
1998 1998
1998 1998
1998. Courier #300 pickup order #11000 at point #41675 at time 273269
300
1999 1999
1999 1999
1999. Courier #300 dropoff order #11000 at point #61675 at time 273294
300
2000 2000
2000 2000
2000. Courier #300 pickup order #11001 at point #41676 at time 273316
300
2001 2001
2001 2001
2001. Courier #300 dropoff order #11001 at po

...

8569 8569
8569 8569
8569. Courier #300 dropoff order #14285 at point #66219 at time 1210498
300
8570 8570
8570 8570
8570. Courier #300 pickup order #14286 at point #46220 at time 1210689
300
8571 8571
8571 8571
8571. Courier #300 dropoff order #14286 at point #66220 at time 1210891
300
8572 8572
8572 8572
8572. Courier #300 pickup order #14287 at point #46221 at time 1210963
300
8573 8573
8573 8573
8573. Courier #300 dropoff order #14287 at point #66221 at time 1211053
300
8574 8574
8574 8574
8574. Courier #300 pickup order #14288 at point #46222 at time 1211187
300
8575 8575
8575 8575
8575. Courier #300 dropoff order #14288 at point #66222 at time 1211292
300
8576 8576
8576 8576
8576. Courier #300 pickup order #14289 at point #46223 at time 1211428
300
8577 8577
8577 8577
8577. Courier #300 dropoff order #14289 at point #66223 at time 1211535
300
8578 8578
8578 8578
8578. Courier #300 pickup order #14290 at point #46224 at time 1211656
300
8579 8579
8579 8579
8579. Courier #300 dropof

9202 9202
9202 9202
9202. Courier #300 pickup order #14602 at point #46536 at time 1306966
300
9203 9203
9203 9203
9203. Courier #300 dropoff order #14602 at point #66536 at time 1307036
300
9204 9204
9204 9204
9204. Courier #300 pickup order #14603 at point #46537 at time 1307131
300
9205 9205
9205 9205
9205. Courier #300 dropoff order #14603 at point #66537 at time 1307176
300
9206 9206
9206 9206
9206. Courier #300 pickup order #14604 at point #46538 at time 1307243
300
9207 9207
9207 9207
9207. Courier #300 dropoff order #14604 at point #66538 at time 1307437
300
9208 9208
9208 9208
9208. Courier #300 pickup order #14605 at point #46539 at time 1307641
300
9209 9209
9209 9209
9209. Courier #300 dropoff order #14605 at point #66539 at time 1307754
300
9210 9210
9210 9210
9210. Courier #300 pickup order #14606 at point #46540 at time 1308040
300
9211 9211
9211 9211
9211. Courier #300 dropoff order #14606 at point #66540 at time 1308206
300
9212 9212
9212 9212
9212. Courier #300 pickup

300
9861 9861
9861 9861
9861. Courier #300 dropoff order #14931 at point #66865 at time 1409084
300
9862 9862
9862 9862
9862. Courier #300 pickup order #14932 at point #46866 at time 1409230
300
9863 9863
9863 9863
9863. Courier #300 dropoff order #14932 at point #66866 at time 1409359
300
9864 9864
9864 9864
9864. Courier #300 pickup order #14933 at point #46867 at time 1409476
300
9865 9865
9865 9865
9865. Courier #300 dropoff order #14933 at point #66867 at time 1409554
300
9866 9866
9866 9866
9866. Courier #300 pickup order #14934 at point #46868 at time 1409754
300
9867 9867
9867 9867
9867. Courier #300 dropoff order #14934 at point #66868 at time 1409767
300
9868 9868
9868 9868
9868. Courier #300 pickup order #14935 at point #46869 at time 1409877
300
9869 9869
9869 9869
9869. Courier #300 dropoff order #14935 at point #66869 at time 1409952
300
9870 9870
9870 9870
9870. Courier #300 pickup order #14936 at point #46870 at time 1410172
300
9871 9871
9871 9871
9871. Courier #300 dr

Order #12859 unfinished
Order #12860 unfinished
Order #12861 unfinished
Order #12862 unassigned
Order #12863 unassigned
Order #12864 unassigned
Order #12865 unfinished
Order #12866 unfinished
Order #12867 unassigned
Order #12868 unfinished
Order #12869 unfinished
Order #12870 unfinished
Order #12871 unfinished
Order #12872 unassigned
Order #12873 unfinished
Order #12874 unfinished
Order #12875 unassigned
Order #12876 unfinished
Order #12877 unassigned
Order #12878 unfinished
Order #12879 unfinished
Order #12880 unfinished
Order #12881 unfinished
Order #12882 unfinished
Order #12883 unfinished
Order #12884 unassigned
Order #12885 unassigned
Order #12886 unfinished
Order #12887 unfinished
Order #12888 unfinished
Order #12889 unfinished
Order #12890 unfinished
Order #12891 unassigned
Order #12892 unfinished
Order #12893 unassigned
Order #12894 unassigned
Order #12895 unfinished
Order #12896 unfinished
Order #12897 unfinished
Order #12898 unassigned
Order #12899 unassigned
Order #12900 una

Order #16656 unfinished
Order #16657 unfinished
Order #16658 unfinished
Order #16659 unfinished
Order #16660 unfinished
Order #16661 unfinished
Order #16662 unfinished
Order #16663 unfinished
Order #16664 unfinished
Order #16665 unfinished
Order #16666 unfinished
Order #16667 unfinished
Order #16668 unfinished
Order #16669 unfinished
Order #16670 unfinished
Order #16671 unfinished
Order #16672 unfinished
Order #16673 unfinished
Order #16674 unfinished
Order #16675 unfinished
Order #16676 unfinished
Order #16677 unfinished
Order #16678 unfinished
Order #16679 unfinished
Order #16680 unfinished
Order #16681 unfinished
Order #16682 unfinished
Order #16683 unfinished
Order #16684 unfinished
Order #16685 unfinished
Order #16686 unfinished
Order #16687 unfinished
Order #16688 unfinished
Order #16689 unfinished
Order #16690 unfinished
Order #16691 unfinished
Order #16692 unfinished
Order #16693 unfinished
Order #16694 unfinished
Order #16695 unfinished
Order #16696 unfinished
Order #16697 unf